In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statistics as stats
from math import *
import random
import datetime
import time

In [8]:
class Pack:
    def __init__(self,ID, tIn): # <--------------
        self.ID = ID
        self.tIn = tIn
        self.t2 = 0
        self.t3 = 0

In [13]:
buffer_registr = [] #буферный регистр, макс. 3 пакета
canals = []
sputnik = []
time_work = 0
time_end = 160 #1*60*1000

N = 4
E = 3

Lambda1 = 0.5
mean2 = 10
sigma2 = 3

In [10]:
def ExponentialDistribution(Lambda):
    x = random.uniform(0,1)
    y = -1 / Lambda * np.log(x)
    return y

def NormalDistribution(mean, sigma):
    x1 = random.uniform(0, 1)
    x2 = random.uniform(0, 1)
    y = sigma * np.cos(2 * pi * x1) * np.sqrt(-2 * np.log(x2)) + mean
    return y

In [11]:
def GetNumPackage():
    num = 0
    while True:
        yield num #генератор
        num += 1

def ConvertToTime(Time):
    dTime = time.gmtime(Time)
    #return str(Time//100//60//60%60) + ' : ' + str(Time//100//60%60) + ' : ' + str(Time//100%60) + ' : ' + str(Time%100)
    return str(Time//100//60%60) + ' : ' + str(Time//100%60) + ' : ' + str(Time%100)


def GetNewPack(TIME): # <--------------
    t1 = ExponentialDistribution(Lambda1)
    t2 = NormalDistribution(mean2, sigma2)
    t3 = random.uniform(8, 12)
    p = Pack(next(genID),TIME+t1)
    p.t2 = t2
    p.t3 = t3
    return p
    
def GetID():
    i = 0
    while True:
        yield i
        i += 1
    
def AddRowToDF(df, dataList, updateColumnsNumList):
    updateIndex = []
    for i in updateColumnsNumList:
        updateIndex.append(df.columns[i])
    s = pd.Series(pd.Series(dataList, index = updateIndex))
    df.fillna('-', inplace = True)
    return df.append(s, ignore_index = True)

In [14]:
tempD = pd.DataFrame(columns=["Общее время работы СМО", "№ пакета", "Состояние пакета", "Размер буфера", 
                              "Занятость каналов", "Пакетов на спутнике", "Всего передано через каналы",
                              "Всего передано через спутник"])

send_pack = True 
genID = GetID()
sk = 0
ss = 0
prevPack = None
while time_work < time_end:
    
    #создаем новый пакет
    pack = GetNewPack(prevPack.tIn if prevPack != None else time_work) # <--------------
    prevPack=pack
    #print(time_work,' : ',pack.ID," | Пакет поступил в СМО...")
    tempD = AddRowToDF(tempD,
                      [ConvertToTime(time_work),
                      pack.ID,
                      "Поступил в СМО",
                      len(buffer_registr),
                      len(canals),
                      len(sputnik),
                      sk, ss],
                      [0, 1, 2, 3, 4, 5, 6, 7])
    
    #пробуем буферизовать пакет
    if len(buffer_registr) < E:
        buffer_registr.append(pack)
        #print(time_work,' : ',pack.ID," | Пакет был помещен в буферную зону... ")
        tempD = AddRowToDF(tempD,
                      [ConvertToTime(time_work),
                      pack.ID,
                      "Помещен в буфер",
                      len(buffer_registr),
                      len(canals),
                      len(sputnik),
                      sk, ss],
                      [0, 1, 2, 3, 4, 5, 6, 7])
    else:
        sputnik.append(pack)
        #print(time_work,' : ',pack.ID," | Пакет был отправлен на спутник... ")
        tempD = AddRowToDF(tempD,
                      [ConvertToTime(time_work),
                      pack.ID,
                      "Отправлен на спутник",
                      len(buffer_registr),
                      len(canals),
                      len(sputnik),
                      sk, ss],
                      [0, 1, 2, 3, 4, 5, 6, 7])

    #передача пакетов из буфера в список каналов
    if len(canals) < N:
        K = min(N - len(canals), len(buffer_registr))
        canals += buffer_registr[:K] # <--------------
        for i in range(K):
            #print(time_work,' : ',buffer_registr[i].ID," | Пакет был удален из буферной зоны... ")
            tempD = AddRowToDF(tempD,
                      [ConvertToTime(time_work),
                      pack.ID,
                      "Удален из буфера",
                      len(buffer_registr),
                      len(canals),
                      len(sputnik),
                      sk, ss],
                      [0, 1, 2, 3, 4, 5, 6, 7])
            buffer_registr.pop(i)
            
        
    #передача пакета(ов) через каналы
    if len(canals) > 0:
        #tym = time.localtime()
        T = time.strftime("%S",time.localtime())
        for p in canals:
            if time_work > p.tIn + p.t2: # <--------------
                p.tIn += p.t2
                sk += 1
                #print(time_work,' : ',p.ID," | Пакет передан через канал под номером ", canals.index(p))
                tempD = AddRowToDF(tempD,
                      [ConvertToTime(time_work),
                      pack.ID,
                      "Передан через канал", #+ str(canals.index(p) + 1),
                      len(buffer_registr),
                      len(canals),
                      len(sputnik),
                      sk, ss],
                      [0, 1, 2, 3, 4, 5, 6, 7])
                canals.remove(p)
    
    #передача пакета через спутник
    if len(sputnik) > 0:
        TT = time.strftime("%S",time.localtime())
        for i in sputnik:
            if send_pack == True:
                pack_sput = sputnik[0]
                send_pack = False
                #print(time_work,' : ',pack_sput.ID," | Начало передачи пакета через спутник... ")
                tempD = AddRowToDF(tempD,
                      [ConvertToTime(time_work),
                      pack.ID,
                      "Начал передачу спутник",
                      len(buffer_registr),
                      len(canals),
                      len(sputnik),
                      sk, ss],
                      [0, 1, 2, 3, 4, 5, 6, 7])
            if time_work > pack_sput.tIn + pack_sput.t3: # <--------------
                #print(time_work,' : ',pack_sput.ID," | Пакет передан по спутнику.. .")
                ss += 1
                tempD = AddRowToDF(tempD,
                      [ConvertToTime(time_work),
                      pack.ID,
                      "Доставлен через спутник",
                      len(buffer_registr),
                      len(canals),
                      len(sputnik),
                      sk, ss],
                      [0, 1, 2, 3, 4, 5, 6, 7])
                sputnik.pop(0)
                send_pack = True
    
    time_work += 1

#pd.set_option("display.max_rows", 500, "display.max_columns", 12)
display(tempD)

itog = pd.DataFrame(columns=['Пакетов передано через КАНАЛЫ','Пакетов передано через СПУТНИК', "Средняя загруженность спутника"])
itog["Пакетов передано через КАНАЛЫ"] = pd.Series([tempD[tempD["Состояние пакета"] == "Передан через канал"]['№ пакета'].count()])
itog["Пакетов передано через СПУТНИК"] = pd.Series([tempD[tempD["Состояние пакета"] == "Доставлен через спутник"]['№ пакета'].count()])
itog["Средняя загруженность спутника"] = pd.Series([tempD[tempD["Состояние пакета"] == "Отправлен на спутник"]['№ пакета'].count()])/itog["Пакетов передано через СПУТНИК"]
display(itog)

,Общее время работы СМО,№ пакета,Состояние пакета,Размер буфера,Занятость каналов,Пакетов на спутнике,Всего передано через каналы,Всего передано через спутник
0,0 : 0 : 0,0,Поступил в СМО,0,0,0,0,0
1,0 : 0 : 0,0,Помещен в буфер,1,0,0,0,0
2,0 : 0 : 0,0,Удален из буфера,1,1,0,0,0
3,0 : 0 : 1,1,Поступил в СМО,0,1,0,0,0
4,0 : 0 : 1,1,Помещен в буфер,1,1,0,0,0
...,...,...,...,...,...,...,...,...
458,0 : 1 : 58,158,Отправлен на спутник,3,4,84,18,50
459,0 : 1 : 59,159,Поступил в СМО,3,4,84,18,50
460,0 : 1 : 59,159,Отправлен на спутник,3,4,85,18,50
461,0 : 1 : 59,159,Доставлен через спутник,3,4,85,18,51


,Пакетов передано через КАНАЛЫ,Пакетов передано через СПУТНИК,Средняя загруженность спутника
0,18,51,2.647059
